<a href="https://colab.research.google.com/github/RMCV-Rajapaksha/TeamInception_Datathon/blob/malitha/task2_eda_and_data_preprocessing_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 2 Notebook for "XGBoost"


## Imports and Setup

**Import the Libararies**


In [31]:
# !pip install tensorflow pandas numpy matplotlib scikit-learn

In [32]:
import pandas as pd
import os
from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

### Connecting to Google Drive

In [33]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Reading data set files

In [34]:
# Developers please update the paths according to were in your google you place datasets
# i recommend making Rootcode-ML folder and in it Datasets folder and placing the csv files there then you wont have to change below code block
bookings_data = pd.read_csv('/content/drive/MyDrive/Rootcode-ML/Datasets/bookings_train.csv')
tasks_data = pd.read_csv('/content/drive/MyDrive/Rootcode-ML/Datasets/tasks.csv')
staffing_data = pd.read_csv('/content/drive/MyDrive/Rootcode-ML/Datasets/staffing_train.csv')

## EDA



### EDA of Staffing data

lets take a look at the columns and some of their values

In [35]:
staffing_data.head()

,date,section_id,employees_on_duty,total_task_time_minutes
0,2021-01-01,SEC-001,1,834.569483
1,2021-01-01,SEC-002,4,1525.368990
2,2021-01-01,SEC-003,2,662.095279
3,2021-01-01,SEC-004,4,1499.112620
4,2021-01-01,SEC-005,3,1238.533073


In [36]:
staffing_data.describe()

,employees_on_duty,total_task_time_minutes
count,5802.000000,5802.000000
mean,4.458807,1662.296769
std,2.794417,1147.134643
min,1.000000,91.109596
25%,3.000000,859.104314
50%,4.000000,1358.176924
75%,6.000000,2123.936358
max,26.000000,10658.173092


removing unnecessary columns. We wont need the total_task_time_minutes because this data wont be provided as input from the test datasets. We need the date and section_id  because they will be to calculate the actual target value (the thing to be predicted) for task 2.

In [37]:
staffing_data = staffing_data.drop(['total_task_time_minutes'],axis=1)

now lets look at the new table headers again.


In [38]:
staffing_data.head()

,date,section_id,employees_on_duty
0,2021-01-01,SEC-001,1
1,2021-01-01,SEC-002,4
2,2021-01-01,SEC-003,2
3,2021-01-01,SEC-004,4
4,2021-01-01,SEC-005,3


lets get a better idea of how null values have been distributed amoung the columns

In [39]:
staffing_data.isna().sum()

,0
date,0
section_id,0
employees_on_duty,0


### EDA of Bookings data

In [40]:
bookings_data.head()

,booking_id,citizen_id,booking_date,appointment_date,appointment_time,check_in_time,check_out_time,task_id,num_documents,queue_number,satisfaction_rating
0,f7b59ba3793fea61282cb02be2247cebd0a75306,4229044420,2021-01-01,2021-01-01,09:03,2021-01-01 09:11:00,2021-01-01 09:48:15.166353269,TASK-002,0,2,4
1,cfe30d53b4b645e4cb6b114f6c9d74a0f25e6a80,5389442635,2021-01-01,2021-01-01,09:12,2021-01-01 09:24:00,2021-01-01 10:24:12.189261137,TASK-001,1,1,4
2,0fb0a5d9b3944dd743a9e7828dc03fa74aa6ebf8,2123783919,2021-01-01,2021-01-01,09:36,2021-01-01 09:29:00,2021-01-01 10:26:48.802260864,TASK-002,1,6,5
3,d4bf826153212fa325ad98b215668ab942365607,6359918966,2021-01-01,2021-01-01,09:45,2021-01-01 10:07:00,2021-01-01 11:00:13.485642822,TASK-001,1,4,4
4,ebb4c9687acbaf65376d8e379bd1341d1c15127e,178732202,2021-01-01,2021-01-01,10:12,2021-01-01 10:26:00,2021-01-01 11:54:53.260180213,TASK-002,1,8,3


In [41]:
bookings_data.describe()

,citizen_id,num_documents,queue_number,satisfaction_rating
count,2.036930e+05,203693.000000,203693.000000,203693.000000
mean,4.996145e+09,2.436790,9.310511,4.187277
std,2.880985e+09,1.734511,8.245711,0.463539
min,4.318100e+04,0.000000,1.000000,3.000000
25%,2.504323e+09,1.000000,3.000000,4.000000
50%,4.995536e+09,2.000000,7.000000,4.000000
75%,7.491514e+09,3.000000,13.000000,4.000000
max,9.999975e+09,14.000000,133.000000,5.000000


In [51]:
bookings_data.isna().sum()

,0
booking_id,0
citizen_id,0
booking_date,0
appointment_date,0
appointment_time,0
check_in_time,6092
check_out_time,6092
task_id,0
num_documents,0
queue_number,0


### EDA of tasks data

In [48]:
tasks_data.head()

,task_id,task_name,section_id,section_name
0,TASK-001,Register new motor vehicles and issue registra...,SEC-001,Vehicle Registration
1,TASK-002,Process the transfer of vehicle ownership.,SEC-001,Vehicle Registration
2,TASK-003,Conduct theoretical and practical examinations...,SEC-002,Driver Licensing
3,TASK-004,Process the renewal and reissue of driving lic...,SEC-002,Driver Licensing
4,TASK-005,Issue international driving permits.,SEC-002,Driver Licensing


In [50]:
tasks_data.describe()

,task_id,task_name,section_id,section_name
count,19,19,19,19
unique,19,19,6,6
top,TASK-001,Register new motor vehicles and issue registra...,SEC-002,Driver Licensing
freq,1,1,4,4


In [49]:
tasks_data.isna().sum()

,0
task_id,0
task_name,0
section_id,0
section_name,0


# Merge Datasets

In [57]:
# --- Merge Bookings with Tasks to get section_id for each booking ---
df_bookings_merged = pd.merge(bookings_data, tasks_data, on='task_id', how='left')


# --- Aggregate Workload Features by day and section ---
# First, ensure appointment_date is a datetime object
df_bookings_merged['appointment_date'] = pd.to_datetime(df_bookings_merged['appointment_date'])

# Group by date and section, then calculate workload
df_workload = df_bookings_merged.groupby(
    [df_bookings_merged['appointment_date'].dt.date, 'section_id']
).agg(
    total_appointments=('booking_id', 'count'),
    total_documents=('num_documents', 'sum')
).reset_index()

# Rename date column for merging
df_workload = df_workload.rename(columns={'appointment_date': 'date'})


# --- Create the final training DataFrame ---
# Ensure date columns in both dataframes are datetime objects for a clean merge
staffing_data['date'] = pd.to_datetime(staffing_data['date'])
df_workload['date'] = pd.to_datetime(df_workload['date'])

# Merge workload data with historical staffing data
training_df = pd.merge(staffing_data, df_workload, on=['date', 'section_id'], how='left')


# --- Handle potential missing values ---
# If a section had staff but no appointments booked that day, workload would be NaN.
# We'll fill these with 0.
training_df['total_appointments'] = training_df['total_appointments'].fillna(0)
training_df['total_documents'] = training_df['total_documents'].fillna(0)

training_df.head()


,date,section_id,employees_on_duty,total_appointments,total_documents
0,2021-01-01,SEC-001,1,13,18
1,2021-01-01,SEC-002,4,35,106
2,2021-01-01,SEC-003,2,21,51
3,2021-01-01,SEC-004,4,32,64
4,2021-01-01,SEC-005,3,23,56


In [58]:
print(f"\nShape of the training data: {training_df.shape}")
print("\nMissing values check:")
training_df.isnull().sum()


Shape of the training data: (5802, 5)

Missing values check:


,0
date,0
section_id,0
employees_on_duty,0
total_appointments,0
total_documents,0


## Data Preprocessing


### Rearrangement and processing of columns

### Dataset seperation according to section_id

In [42]:
task_counts = staffing_data.groupby("section_id").size().reset_index(name="count")
task_counts

,section_id,count
0,SEC-001,967
1,SEC-002,967
2,SEC-003,967
3,SEC-004,967
4,SEC-005,967
5,SEC-006,967


In [43]:
sec_1_data = staffing_data[staffing_data['section_id'] == 'SEC-001'].copy()
sec_2_data = staffing_data[staffing_data['section_id'] == 'SEC-002'].copy()
sec_3_data = staffing_data[staffing_data['section_id'] == 'SEC-003'].copy()
sec_4_data = staffing_data[staffing_data['section_id'] == 'SEC-004'].copy()
sec_5_data = staffing_data[staffing_data['section_id'] == 'SEC-005'].copy()
sec_6_data = staffing_data[staffing_data['section_id'] == 'SEC-006'].copy()


In [52]:
sec_4_data.head()

,date,section_id,employees_on_duty
3,2021-01-01,SEC-004,4
9,2021-01-04,SEC-004,6
15,2021-01-05,SEC-004,4
21,2021-01-06,SEC-004,1
27,2021-01-07,SEC-004,3
